# ETL - Extração, Transformação e Carregamento de Dados no BigQuery

Este notebook realiza o processo completo de ETL (Extract, Transform, Load) para carregar dados de diferentes fontes externas no Google BigQuery.

## Visão Geral do Processo

O pipeline processa quatro datasets principais:
1. **Pedidos**: Dados de transações em formato JSON comprimido
2. **Consumidores**: Informações de usuários em CSV comprimido
3. **Restaurantes**: Dados de estabelecimentos em CSV comprimido
4. **Teste A/B**: Dados de experimentos em arquivo TAR comprimido

## Estrutura do Notebook

1. **Instalação de Dependências**: Configuração do ambiente
2. **Autenticação**: Acesso ao Google Cloud Platform
3. **Configuração Inicial**: Imports e parâmetros do projeto
4. **ETL Pedidos**: Processamento em chunks para otimização de memória
5. **ETL Consumidores**: Carregamento direto de dados estruturados
6. **ETL Restaurantes**: Processamento de dados de estabelecimentos
7. **ETL Teste A/B**: Extração e carregamento de dados experimentais
8. **Validação**: Verificação dos dados carregados

---

## 1. Instalação de Dependências

Instalamos os pacotes necessários para o processo de ETL:

- **pandas**: Manipulação e análise de dados
- **pandas-gbq**: Interface simplificada para BigQuery
- **pyarrow**: Engine otimizado para operações com DataFrames

Estes pacotes fornecem as ferramentas essenciais para extração, transformação e carregamento eficiente dos dados.

In [1]:
!pip install --quiet pandas pandas-gbq pyarrow

## 2. Autenticação no Google Cloud

A autenticação é necessária para acessar e escrever dados no BigQuery. O Google Colab fornece uma interface simplificada que gerencia automaticamente as credenciais.

In [2]:
from google.colab import auth
auth.authenticate_user()
print("Autenticação realizada com sucesso!")

Autenticação realizada com sucesso!


## 3. Configuração Inicial

Importamos as bibliotecas necessárias e configuramos os parâmetros do projeto:

- **pandas**: Manipulação de dados
- **urllib.request**: Download de arquivos
- **gzip/tarfile**: Descompressão de arquivos
- **google.cloud.bigquery**: Cliente nativo do BigQuery
- **pandas_gbq**: Interface pandas para BigQuery

In [3]:
import pandas as pd
import urllib.request
import gzip
import tarfile
from google.cloud import bigquery
import pandas_gbq
import os
from datetime import datetime

# Configurações do projeto
project_id = "294185590533"
dataset_id = "ifood"
client = bigquery.Client(project=project_id)

print(f"Projeto configurado: {project_id}")
print(f"Dataset de destino: {dataset_id}")
print(f"Timestamp de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Projeto configurado: 294185590533
Dataset de destino: ifood
Timestamp de execução: 2025-07-03 19:12:26


## 4. Funções Auxiliares

Definimos funções reutilizáveis para otimizar o processo de ETL e fornecer feedback detalhado sobre o progresso.

In [4]:
def download_arquivo(url, nome_arquivo):
    """
    Faz download de um arquivo da URL especificada

    Args:
        url (str): URL do arquivo para download
        nome_arquivo (str): Nome local para salvar o arquivo
    """
    print(f"Iniciando download: {nome_arquivo}")
    urllib.request.urlretrieve(url, nome_arquivo)

    # Verifica o tamanho do arquivo baixado
    tamanho_mb = os.path.getsize(nome_arquivo) / (1024 * 1024)
    print(f"Download concluído: {nome_arquivo} ({tamanho_mb:.2f} MB)")


def carregar_para_bigquery(dataframe, tabela_destino, modo="replace"):
    """
    Carrega um DataFrame para o BigQuery com tratamento de erros

    Args:
        dataframe (pd.DataFrame): DataFrame a ser carregado
        tabela_destino (str): Nome da tabela de destino
        modo (str): Modo de carregamento ('replace' ou 'append')
    """
    try:
        pandas_gbq.to_gbq(
            dataframe,
            destination_table=f"{dataset_id}.{tabela_destino}",
            project_id=project_id,
            if_exists=modo
        )
        print(f"Sucesso: {len(dataframe):,} registros carregados em {tabela_destino}")
    except Exception as e:
        print(f"Erro ao carregar {tabela_destino}: {str(e)}")
        raise


def verificar_tabela(nome_tabela):
    """
    Verifica informações básicas de uma tabela no BigQuery

    Args:
        nome_tabela (str): Nome da tabela para verificação
    """
    query = f"""
    SELECT
        COUNT(*) as total_registros,
        COUNT(DISTINCT *) as registros_unicos
    FROM `{project_id}.{dataset_id}.{nome_tabela}`
    """

    try:
        resultado = client.query(query).to_dataframe()
        total = resultado.iloc[0]['total_registros']
        unicos = resultado.iloc[0]['registros_unicos']
        print(f"Tabela {nome_tabela}: {total:,} registros ({unicos:,} únicos)")
        return total, unicos
    except Exception as e:
        print(f"Erro ao verificar {nome_tabela}: {str(e)}")
        return None, None

## 5. ETL - Dados de Pedidos (order.json.gz)

Os dados de pedidos são o maior dataset e requerem processamento especial:

### Características dos Dados
- **Formato**: JSON Lines comprimido com gzip
- **Volume**: Aproximadamente 3.67 milhões de registros
- **Estratégia**: Processamento em chunks para otimização de memória

### Processamento em Chunks
- **Tamanho do chunk**: 100.000 registros por vez
- **Vantagem**: Evita sobrecarga de memória
- **Modo de carregamento**: Replace no primeiro chunk, append nos demais

In [5]:
# URL e configurações para dados de pedidos
orders_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
orders_file = "order.json.gz"
orders_table = "orders_raw"

# Download do arquivo
download_arquivo(orders_url, orders_file)

Iniciando download: order.json.gz
Download concluído: order.json.gz (1536.18 MB)


In [6]:
# Processamento em chunks para otimização de memória
chunksize = 100_000
total_registros = 0

print(f"Iniciando processamento de {orders_file} em chunks de {chunksize:,} registros")
print("=" * 60)

with gzip.open(orders_file, "rt", encoding="utf-8") as f:
    reader = pd.read_json(f, lines=True, chunksize=chunksize)

    for i, chunk in enumerate(reader):
        # Informações do chunk atual
        chunk_size = len(chunk)
        total_registros += chunk_size

        print(f"Processando chunk {i+1}: {chunk_size:,} registros")

        # Carrega para BigQuery
        modo_carregamento = "replace" if i == 0 else "append"
        carregar_para_bigquery(chunk, orders_table, modo_carregamento)

        print(f"Total acumulado: {total_registros:,} registros")
        print("-" * 40)

print("=" * 60)
print(f"Processamento de pedidos concluído: {total_registros:,} registros totais")

Iniciando processamento de order.json.gz em chunks de 100,000 registros
Processando chunk 1: 100,000 registros


100%|██████████| 1/1 [00:00<00:00, 471.85it/s]


Sucesso: 100,000 registros carregados em orders_raw
Total acumulado: 100,000 registros
----------------------------------------
Processando chunk 2: 100,000 registros


100%|██████████| 1/1 [00:00<00:00, 4415.06it/s]


Sucesso: 100,000 registros carregados em orders_raw
Total acumulado: 200,000 registros
----------------------------------------
Processando chunk 3: 100,000 registros


...

Sucesso: 100,000 registros carregados em orders_raw
Total acumulado: 3,500,000 registros
----------------------------------------
Processando chunk 36: 100,000 registros


100%|██████████| 1/1 [00:00<00:00, 2208.69it/s]


Sucesso: 100,000 registros carregados em orders_raw
Total acumulado: 3,600,000 registros
----------------------------------------
Processando chunk 37: 70,826 registros


100%|██████████| 1/1 [00:00<00:00, 2439.97it/s]

Sucesso: 70,826 registros carregados em orders_raw
Total acumulado: 3,670,826 registros
----------------------------------------
Processamento de pedidos concluído: 3,670,826 registros totais


## 6. ETL - Dados de Consumidores (consumer.csv.gz)

Os dados de consumidores são estruturados e podem ser processados diretamente:

### Características dos Dados
- **Formato**: CSV comprimido com gzip
- **Volume**: Dataset menor, carregamento direto
- **Estratégia**: Carregamento completo em memória

In [7]:
# Configurações para dados de consumidores
consumers_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
consumers_file = "consumer.csv.gz"
consumers_table = "consumers_raw"

print("Processando dados de consumidores")
print("=" * 40)

# Download do arquivo
download_arquivo(consumers_url, consumers_file)

# Carregamento e processamento
print("Carregando dados em memória...")
consumers_df = pd.read_csv(consumers_file)

# Informações do dataset
print(f"Dados carregados: {len(consumers_df):,} registros x {len(consumers_df.columns)} colunas")
print(f"Colunas: {list(consumers_df.columns)}")

# Carrega para BigQuery
carregar_para_bigquery(consumers_df, consumers_table)

print("Processamento de consumidores concluído")

Processando dados de consumidores
Iniciando download: consumer.csv.gz
Download concluído: consumer.csv.gz (46.25 MB)
Carregando dados em memória...
Dados carregados: 806,156 registros x 7 colunas
Colunas: ['customer_id', 'language', 'created_at', 'active', 'customer_name', 'customer_phone_area', 'customer_phone_number']


100%|██████████| 1/1 [00:00<00:00, 8848.74it/s]

Sucesso: 806,156 registros carregados em consumers_raw
Processamento de consumidores concluído


## 7. ETL - Dados de Restaurantes (restaurant.csv.gz)

Os dados de restaurantes seguem o mesmo padrão dos consumidores:

### Características dos Dados
- **Formato**: CSV comprimido com gzip
- **Volume**: Dataset estruturado de tamanho moderado
- **Estratégia**: Carregamento direto otimizado

In [8]:
# Configurações para dados de restaurantes
restaurants_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
restaurants_file = "restaurant.csv.gz"
restaurants_table = "restaurants_raw"

print("Processando dados de restaurantes")
print("=" * 40)

# Download do arquivo
download_arquivo(restaurants_url, restaurants_file)

# Carregamento e processamento
print("Carregando dados em memória...")
restaurants_df = pd.read_csv(restaurants_file)

# Informações do dataset
print(f"Dados carregados: {len(restaurants_df):,} registros x {len(restaurants_df.columns)} colunas")
print(f"Colunas: {list(restaurants_df.columns)}")

# Carrega para BigQuery
carregar_para_bigquery(restaurants_df, restaurants_table)

print("Processamento de restaurantes concluído")

Processando dados de restaurantes
Iniciando download: restaurant.csv.gz
Download concluído: restaurant.csv.gz (0.38 MB)
Carregando dados em memória...
Dados carregados: 7,292 registros x 12 colunas
Colunas: ['id', 'created_at', 'enabled', 'price_range', 'average_ticket', 'takeout_time', 'delivery_time', 'minimum_order_value', 'merchant_zip_code', 'merchant_city', 'merchant_state', 'merchant_country']


100%|██████████| 1/1 [00:00<00:00, 9446.63it/s]

Sucesso: 7,292 registros carregados em restaurants_raw
Processamento de restaurantes concluído


## 8. ETL - Dados de Teste A/B (ab_test_ref.tar.gz)

Os dados de teste A/B requerem extração adicional devido ao formato TAR:

### Características dos Dados
- **Formato**: CSV dentro de arquivo TAR comprimido
- **Processo**: Download → Extração → Carregamento
- **Estratégia**: Extração completa e carregamento direto

In [9]:
# Configurações para dados de teste A/B
ab_test_url = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
ab_test_file = "ab_test_ref.tar.gz"
ab_test_csv = "ab_test_ref.csv"
ab_test_table = "ab_test_ref_raw"

print("Processando dados de teste A/B")
print("=" * 40)

# Download do arquivo
download_arquivo(ab_test_url, ab_test_file)

# Extração do arquivo TAR
print("Extraindo arquivo TAR...")
with tarfile.open(ab_test_file, "r:gz") as tar:
    tar.extractall()
    print(f"Arquivos extraídos: {tar.getnames()}")

# Verificação do arquivo extraído
if os.path.exists(ab_test_csv):
    tamanho_csv = os.path.getsize(ab_test_csv) / (1024 * 1024)
    print(f"Arquivo CSV extraído: {ab_test_csv} ({tamanho_csv:.2f} MB)")
else:
    print(f"Erro: Arquivo {ab_test_csv} não encontrado após extração")
    raise FileNotFoundError(f"Arquivo {ab_test_csv} não foi extraído corretamente")

# Carregamento e processamento
print("Carregando dados em memória...")
ab_test_df = pd.read_csv(ab_test_csv)

# Informações do dataset
print(f"Dados carregados: {len(ab_test_df):,} registros x {len(ab_test_df.columns)} colunas")
print(f"Colunas: {list(ab_test_df.columns)}")

# Carrega para BigQuery
carregar_para_bigquery(ab_test_df, ab_test_table)

print("Processamento de teste A/B concluído")

Processando dados de teste A/B
Iniciando download: ab_test_ref.tar.gz
Download concluído: ab_test_ref.tar.gz (29.78 MB)
Extraindo arquivo TAR...
Arquivos extraídos: ['._ab_test_ref.csv', 'ab_test_ref.csv']
Arquivo CSV extraído: ab_test_ref.csv (55.72 MB)
Carregando dados em memória...
Dados carregados: 806,467 registros x 2 colunas
Colunas: ['customer_id', 'is_target']


100%|██████████| 1/1 [00:00<00:00, 2576.35it/s]

Sucesso: 806,467 registros carregados em ab_test_ref_raw
Processamento de teste A/B concluído


## 9. Validação dos Dados Carregados

Verificamos se todos os dados foram carregados corretamente no BigQuery, incluindo contagens de registros e verificações de integridade.

In [10]:
print("VALIDAÇÃO DOS DADOS CARREGADOS")
print("=" * 50)

# Lista de todas as tabelas criadas
tabelas_criadas = [
    "orders_raw",
    "consumers_raw",
    "restaurants_raw",
    "ab_test_ref_raw"
]

# Verifica cada tabela
resultados_validacao = {}

for tabela in tabelas_criadas:
    print(f"Verificando tabela: {tabela}")
    total, unicos = verificar_tabela(tabela)
    resultados_validacao[tabela] = {'total': total, 'unicos': unicos}
    print("-" * 30)

print("=" * 50)

# Resumo consolidado
print("RESUMO FINAL")
print("=" * 20)

total_geral = 0
for tabela, dados in resultados_validacao.items():
    if dados['total'] is not None:
        total_geral += dados['total']
        print(f"{tabela}: {dados['total']:,} registros")
    else:
        print(f"{tabela}: ERRO na verificação")

print("-" * 20)
print(f"Total geral: {total_geral:,} registros")
print(f"Timestamp de conclusão: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

VALIDAÇÃO DOS DADOS CARREGADOS
Verificando tabela: orders_raw
Erro ao verificar orders_raw: 400 COUNT(*) cannot be used with DISTINCT at [4:9]; reason: invalidQuery, location: query, message: COUNT(*) cannot be used with DISTINCT at [4:9]

Location: US
Job ID: 73048739-0408-4d9c-a0b9-44f37f52be7f

------------------------------
Verificando tabela: consumers_raw
Erro ao verificar consumers_raw: 400 COUNT(*) cannot be used with DISTINCT at [4:9]; reason: invalidQuery, location: query, message: COUNT(*) cannot be used with DISTINCT at [4:9]

Location: US
Job ID: 3ee36049-f45f-469a-a59f-74165a763db5

------------------------------
Verificando tabela: restaurants_raw
Erro ao verificar restaurants_raw: 400 COUNT(*) cannot be used with DISTINCT at [4:9]; reason: invalidQuery, location: query, message: COUNT(*) cannot be used with DISTINCT at [4:9]

Location: US
Job ID: ec927e99-b9d2-4f5f-beee-359f484a3943

------------------------------
Verificando tabela: ab_test_ref_raw
Erro ao verificar ab

## 10. Limpeza de Arquivos Temporários

Remove os arquivos baixados para liberar espaço em disco, mantendo apenas os dados no BigQuery.

In [11]:
# Lista de arquivos para limpeza
arquivos_temporarios = [
    "order.json.gz",
    "consumer.csv.gz",
    "restaurant.csv.gz",
    "ab_test_ref.tar.gz",
    "ab_test_ref.csv"
]

print("Limpeza de arquivos temporários")
print("=" * 35)

espaco_liberado = 0

for arquivo in arquivos_temporarios:
    if os.path.exists(arquivo):
        tamanho = os.path.getsize(arquivo) / (1024 * 1024)  # MB
        espaco_liberado += tamanho
        os.remove(arquivo)
        print(f"Removido: {arquivo} ({tamanho:.2f} MB)")
    else:
        print(f"Não encontrado: {arquivo}")

print("-" * 35)
print(f"Espaço total liberado: {espaco_liberado:.2f} MB")
print("Limpeza concluída")

Limpeza de arquivos temporários
Removido: order.json.gz (1536.18 MB)
Removido: consumer.csv.gz (46.25 MB)
Removido: restaurant.csv.gz (0.38 MB)
Removido: ab_test_ref.tar.gz (29.78 MB)
Removido: ab_test_ref.csv (55.72 MB)
-----------------------------------
Espaço total liberado: 1668.32 MB
Limpeza concluída


## 11. Instruções de Uso e Próximos Passos

### Dados Disponíveis no BigQuery

Após a execução bem-sucedida deste notebook, os seguintes datasets estarão disponíveis:

1. **`orders_raw`**: Dados de pedidos (~3.67M registros)
2. **`consumers_raw`**: Informações de consumidores
3. **`restaurants_raw`**: Dados de restaurantes
4. **`ab_test_ref_raw`**: Dados de experimentos A/B

### Como Acessar os Dados

```sql
-- Exemplo de consulta
SELECT COUNT(*) as total_pedidos
FROM `294185590533.ifood.orders_raw`
```

### Próximos Passos Recomendados

1. **Análise Exploratória**: Use o notebook de EDA para entender os dados
2. **Modelagem de Dados**: Crie tabelas dimensionais e de fatos
3. **Análises de Negócio**: Desenvolva métricas e KPIs
4. **Automação**: Configure pipelines para atualizações regulares

### Monitoramento

- Verifique regularmente a integridade dos dados
- Monitore o crescimento das tabelas
- Implemente alertas para falhas no pipeline